In [16]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error


ImportError: cannot import name 'root_mean_squared_error' from 'sklearn.metrics' (/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/metrics/__init__.py)

In [3]:
pd.__version__

'1.4.2'

In [4]:
sklearn.__version__

'1.0.2'

In [5]:
!pip install pyarrow

In [4]:
df_jan_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [23]:
df_jan_2023.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [24]:
df_jan_2023.shape[1]

19

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [6]:
duration_diff_sr = df_jan_2023.tpep_dropoff_datetime - df_jan_2023.tpep_pickup_datetime
df_jan_2023['duration'] = duration_diff_sr.apply(lambda x: x.total_seconds()/60)

In [40]:
print(f'''The standard deviation in minutes of the ride duration is {np.round(np.std(df_jan_2023['duration']), 2)}.''')


The standard deviation in minutes of the ride duration is 42.59.


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [41]:
duration_df_without_outliers = df_jan_2023['duration'][(df_jan_2023['duration'] >= 1) & (df_jan_2023['duration'] <= 60)]
duration_df_with_outliers = df_jan_2023['duration']
np.round(100*duration_df_without_outliers.shape[0]/duration_df_with_outliers.shape[0])



98.0

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [7]:
jan_wo_df = df_jan_2023[(df_jan_2023['duration'] >= 1) & (df_jan_2023['duration'] <= 60)]
jan_wo_pu_do_loc_df = jan_wo_df[['PULocationID', 'DOLocationID']].astype(str)


In [8]:
jan_wo_pu_do_loc_dict = jan_wo_pu_do_loc_df.to_dict(orient='records')


In [9]:
dv = DictVectorizer()


In [10]:
jan_wo_transf_df = dv.fit_transform(jan_wo_pu_do_loc_dict)
print(f'The dimensions of the feature matrix is {jan_wo_transf_df.shape[1]}.')

The dimensions of the feature matrix is 515.




## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [11]:
X_jan = jan_wo_transf_df.copy()
y_jan = jan_wo_df['duration'].copy()

In [12]:
lr = LinearRegression()
lr.fit(X_jan, y_jan)

LinearRegression()

In [13]:
y_jan_pred = lr.predict(X_jan)
print(f'The RMSE on the January data is: {round(root_mean_squared_error(y_jan, y_jan_pred), 2)}.')

NameError: name 'root_mean_squared_error' is not defined

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [ ]:
feb_duration_diff_sr = yt_feb23_df.tpep_dropoff_datetime - yt_feb23_df.tpep_pickup_datetime
yt_feb23_df['duration'] = feb_duration_diff_sr.apply(lambda x: x.total_seconds()/60)

In [ ]:
feb_wo_df = yt_feb23_df[(yt_feb23_df['duration'] >= 1) & (yt_feb23_df['duration'] <= 60)]
feb_wo_pu_do_loc_df = feb_wo_df[['PULocationID', 'DOLocationID']].astype(str)

In [ ]:
feb_wo_pu_do_loc_dict = feb_wo_pu_do_loc_df.to_dict(orient='records')
